In [1]:
import langchain

In [2]:
#setup API key

f = open('/content/gt.txt')
key = f.read()

LangChain is an open-source framework that acts like a toolkit and orchestration layer for building applications with Large Language Models.

It simplifies the process of chaining together different components (like LLMs, data sources, and tools) to create complex, intelligent applications. It's designed to make prompt engineering more efficient and allow developers to adapt language models to specific business contexts.


In [3]:
!pip install langchain-google-genai
from langchain_google_genai import ChatGoogleGenerativeAI

chat_model = ChatGoogleGenerativeAI(api_key = key, model = "gemini-2.5-flash", temperature=1)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 2.8 MB/s eta 0:00:00


In [4]:
PROMPT = "what is LangChain? Explain in 200 words."

model_response = chat_model.invoke(PROMPT)
model_response

AIMessage(content='LangChain is an open-source framework designed to simplify the development of applications powered by large language models (LLMs). It bridges the gap between raw LLMs and complex, real-world use cases. While LLMs are powerful, they often lack memory, external data access, and the ability to interact with tools.\n\nLangChain provides modular components to connect LLMs with external data sources and other computational steps. Its core idea revolves around "chains" and "agents." **Chains** allow you to sequence multiple LLM calls or other utilities, like fetching information from databases, summarizing documents, or processing text in steps. **Agents** empower LLMs to reason and decide which tools to use dynamically to achieve a specific goal, such as searching the web, performing calculations, or interacting with APIs.\n\nIt simplifies managing prompts, handling conversational memory, and integrating diverse data types (e.g., PDFs, websites) with vector databases.\n\n

In [5]:
#OUTPUT PARSING: LLMs primarily generate free-form text. However, in many applications, you need structured data (e.g., a list of items, a JSON object, a specific format). OutputParsers bridge this gap.

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(model_response)

'LangChain is an open-source framework designed to simplify the development of applications powered by large language models (LLMs). It bridges the gap between raw LLMs and complex, real-world use cases. While LLMs are powerful, they often lack memory, external data access, and the ability to interact with tools.\n\nLangChain provides modular components to connect LLMs with external data sources and other computational steps. Its core idea revolves around "chains" and "agents." **Chains** allow you to sequence multiple LLM calls or other utilities, like fetching information from databases, summarizing documents, or processing text in steps. **Agents** empower LLMs to reason and decide which tools to use dynamically to achieve a specific goal, such as searching the web, performing calculations, or interacting with APIs.\n\nIt simplifies managing prompts, handling conversational memory, and integrating diverse data types (e.g., PDFs, websites) with vector databases.\n\nUltimately, LangCh

In [6]:
#Prompt Template: two types- PromptTemplate and ChatPromptTemplate
#Think of them as blueprints for your prompt templates.

In [7]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate(
    messages = [
    ("system", "Your are helpful AI tutor with expertise in Data Science and Artificial Intelligence"),
    ("human", "What is {topic}? Explain in 200 words."),
    ]
)



In [8]:
prompt_template.input_variables

['topic']

In [9]:
prompt_template.invoke({"topic": "LangChain"})

ChatPromptValue(messages=[SystemMessage(content='Your are helpful AI tutor with expertise in Data Science and Artificial Intelligence', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is LangChain? Explain in 200 words.', additional_kwargs={}, response_metadata={})])

In [10]:
print(prompt_template.invoke({"topic": "LangChain"}).to_string())

System: Your are helpful AI tutor with expertise in Data Science and Artificial Intelligence
Human: What is LangChain? Explain in 200 words.


In [11]:
"""
CHAIN (|)- A chain is simply a sequence of operations where the output of one step becomes the input of the next. This allows you to build more complex and intelligent workflows than a single LLM call could achieve.

The `|` symbol chains together the different components feeds the output from one component as input into the next component.
"""
#1 Prompt Template
prompt =  prompt_template.invoke({"topic":"Linear Regression"})
print(type(prompt))

#2 Chat Model
model_response = chat_model.invoke(prompt)
print(type(model_response))

#3 output Parser
final_output = output_parser.invoke(model_response)
print(type(final_output))

<class 'langchain_core.prompt_values.ChatPromptValue'>
<class 'langchain_core.messages.ai.AIMessage'>
<class 'str'>


In [12]:
chain = prompt_template | chat_model | output_parser

user_input = {"topic": "NLP"}

chain.invoke(user_input)

'Natural Language Processing (NLP) is a dynamic field within Artificial Intelligence (AI) that focuses on enabling computers to understand, interpret, and generate human language. Its primary goal is to bridge the communication gap between humans and machines, allowing computers to process and make sense of text and spoken words.\n\nNLP combines computational linguistics (rule-based modeling of language) with machine learning and deep learning techniques. This allows machines to "read" text, "hear" speech, decipher its meaning, extract information, determine sentiment, and even respond in a natural, human-like manner.\n\nKey applications of NLP are ubiquitous, including:\n*   **Machine Translation:** Automatically converting text from one language to another (e.g., Google Translate).\n*   **Sentiment Analysis:** Identifying the emotional tone or opinion expressed in text (e.g., customer reviews).\n*   **Chatbots and Virtual Assistants:** Powering conversational interfaces like Siri, Al

In [13]:
#Building a CHAT PROMPT TEMPLATE - Create an AI Tutor App that uses Prompts and Chat internally to give Python Implementation tutorial for Data Science topics

from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

#Constructing System Prompt
system_prompt = SystemMessagePromptTemplate.from_template("""
You are friendly AI tutor with expertise in Data Science and AI who tells step by step python implementation.
""")

#Constructing Human Prompt
human_prompt = HumanMessagePromptTemplate.from_template("Tell me a python implementation for {topic_name}.")

#Compiling CHAT PROMPT
chat_prompt = ChatPromptTemplate(messages=[system_prompt, human_prompt])

chat_prompt

ChatPromptTemplate(input_variables=['topic_name'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='\nYou are friendly AI tutor with expertise in Data Science and AI who tells step by step python implementation.\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic_name'], input_types={}, partial_variables={}, template='Tell me a python implementation for {topic_name}.'), additional_kwargs={})])

In [14]:
#Building a CHAT MODEL
from langchain_google_genai import ChatGoogleGenerativeAI

#pass the standard parameters during initialization
chat_model = ChatGoogleGenerativeAI(api_key = key, model="gemini-2.5-flash", temperature = 1)
#print(key)

In [15]:
#Building an OUPUT PARSER
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [16]:
#CHAINING all components together

chain = chat_prompt | chat_model | output_parser

In [17]:
#calling the CHAIN
user_input = {"topic_name": "Classifiers in Python"}

output = chain.invoke(user_input)
print(output)

Hey there! 👋

You're looking to dive into Classifiers in Python – that's a fantastic area of Data Science! Classifiers are machine learning models used to categorize data into one of several predefined classes. Think of it like sorting emails into "Spam" or "Not Spam," or identifying if an image contains a "Cat" or "Dog."

The go-to library for this in Python is **scikit-learn (sklearn)**. It provides a consistent API for a wide range of classification algorithms.

Let's walk through a step-by-step implementation. We'll cover:
1.  **Generating / Loading Data**: We need data to classify!
2.  **Splitting Data**: Training and testing sets.
3.  **Feature Scaling (Optional but Recommended)**: Standardizing our data.
4.  **Choosing and Training a Classifier**: Fitting the model to our training data.
5.  **Making Predictions**: Using the trained model on unseen data.
6.  **Evaluating the Classifier**: How well did it do?

We'll demonstrate with a few common classifiers:
*   **Logistic Regress